In [1]:
import os, cv2, random
import numpy as np
import pandas as pd
from keras.utils.np_utils import to_categorical
import shutil
#import matplotlib.pyplot as plt
#from matplotlib import ticker
#import seaborn as sns
#%matplotlib inline 

Using TensorFlow backend.


# Input Data

In [2]:
prefix_dir = 'input/train/'

ROWS = 64
COLS = 64
CHANNELS = 3

images = [i for i in os.listdir(prefix_dir)] # use this for full dataset
dogs =   [i for i in os.listdir(prefix_dir) if 'dog' in i]
cats =   [i for i in os.listdir(prefix_dir) if 'cat' in i]

In [3]:
def get_data(left_perc,train_perc):
    random.shuffle(images)
    left_images = images[:int(len(images)*left_perc)]
    train_images = left_images[:int(len(left_images)*train_perc)]
    test_images = left_images[int(len(left_images)*train_perc):]
    random.shuffle(cats)
    random.shuffle(dogs)
    left_cats = cats[:int(len(cats)*left_perc)]
    left_dogs = dogs[:int(len(dogs)*left_perc)]
    train_num = int(len(left_cats)*train_perc)
    train_cats = left_cats[:train_num]
    train_dogs = left_dogs[:train_num]
    test_cats = left_cats[train_num:]
    test_dogs = left_dogs[train_num:]
    train_images = train_cats+train_dogs
    test_images = test_cats+test_dogs
    return train_images,test_images

def read_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_COLOR) #cv2.IMREAD_GRAYSCALE
    img = img[:,:,::-1]/255
    return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)

def prep_data(images,prefix_dir):
    count = len(images)
    data = np.ndarray((count,ROWS,COLS,CHANNELS))
    for i, image_file in enumerate(images):
        image = read_image(prefix_dir+image_file)
        data[i] = image
        if (i+1)%2000 == 0: print('Processed {} of {}'.format(i+1, count))
    return data

In [4]:
# We just use [left_perc] of our total dataset.
left_perc = 0.2
train_perc = 0.8

train_images,test_images = get_data(left_perc,train_perc)
train_x = prep_data(train_images,prefix_dir)
test_x = prep_data(test_images,prefix_dir)

train_y = [1 if x[:3]=='dog' else 0 for x in train_images]
test_y = [1 if x[:3]=='dog' else 0 for x in test_images]
train_y = to_categorical(train_y)
test_y = to_categorical(test_y)

Processed 2000 of 4000
Processed 4000 of 4000


In [4]:
print("train_x shape: {}".format(train_x.shape))
print("test_x shape: {}".format(test_x.shape))
print('train_y shape: {}'.format(train_y.shape))
print('test_y shape: {}'.format(test_y.shape))

train_x shape: (4000, 64, 64, 3)
test_x shape: (1000, 64, 64, 3)
train_y shape: (4000, 2)
test_y shape: (1000, 2)


# Set up 4 Models

In [6]:
from keras.models import Sequential
from keras.models import Model  
from keras.layers import Dense,Flatten,Dropout,Input,concatenate,BatchNormalization,Activation,ZeroPadding2D,add
from keras.layers.convolutional import Conv2D,MaxPooling2D  
  
from keras.layers.convolutional import Conv2D,MaxPooling2D,AveragePooling2D  
from keras.optimizers import SGD  
import numpy as np  
seed = 888 
np.random.seed(seed)

### Model1: LeNet

In [7]:
model1 = Sequential()  
model1.add(Conv2D(32,(5,5),strides=(1,1),input_shape=(64,64,3),padding='valid',activation='relu',kernel_initializer='uniform')) 
model1.add(MaxPooling2D(pool_size=(2,2)))  
model1.add(Conv2D(64,(5,5),strides=(1,1),padding='valid',activation='relu',kernel_initializer='uniform'))  
model1.add(MaxPooling2D(pool_size=(2,2)))  
model1.add(Flatten())  
model1.add(Dense(100,activation='relu'))  
model1.add(Dense(2,activation='softmax'))  
model1.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])  
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 60, 60, 32)        2432      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 10816)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               1081700   
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total para

In [8]:
model1.fit(train_x,train_y,batch_size=50,epochs=20,verbose=1,validation_data=(test_x,test_y))
model1.save('model1.h5')

Train on 4000 samples, validate on 1000 samples
Epoch 1/20
4000/4000 [==============================] - 41s 10ms/step - loss: 0.6921 - acc: 0.5203 - val_loss: 0.6883 - val_acc: 0.5080
Epoch 2/20
4000/4000 [==============================] - 41s 10ms/step - loss: 0.6866 - acc: 0.5520 - val_loss: 0.6835 - val_acc: 0.5210
Epoch 3/20
4000/4000 [==============================] - 42s 10ms/step - loss: 0.6823 - acc: 0.5705 - val_loss: 0.6756 - val_acc: 0.6170
Epoch 4/20
4000/4000 [==============================] - 45s 11ms/step - loss: 0.6765 - acc: 0.5800 - val_loss: 0.6722 - val_acc: 0.5590
Epoch 5/20
4000/4000 [==============================] - 48s 12ms/step - loss: 0.6744 - acc: 0.5773 - val_loss: 0.6656 - val_acc: 0.5960
Epoch 6/20
4000/4000 [==============================] - 44s 11ms/step - loss: 0.6674 - acc: 0.5910 - val_loss: 0.6549 - val_acc: 0.6360
Epoch 7/20
4000/4000 [==============================] - 46s 11ms/step - loss: 0.6594 - acc: 0.6123 - val_loss: 0.6672 - val_acc: 0.5750


### Model2: AlexNet

In [9]:
model2 = Sequential()  
#model2.add(Conv2D(96,(11,11),strides=(4,4),input_shape=(227,227,3),padding='valid',activation='relu',kernel_initializer='uniform'))
model2.add(Conv2D(96,(11,11),strides=(2,2),input_shape=(64,64,3),padding='valid',activation='relu',kernel_initializer='uniform'))
model2.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))  
model2.add(Conv2D(256,(5,5),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model2.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))  
model2.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model2.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model2.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model2.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))  
model2.add(Flatten())  
model2.add(Dense(4096,activation='relu'))  
model2.add(Dropout(0.5))  
model2.add(Dense(4096,activation='relu'))  
model2.add(Dropout(0.5))  
model2.add(Dense(2,activation='softmax'))  
model2.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])  
model2.summary()  

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 27, 27, 96)        34944     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 96)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 256)       614656    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 6, 6, 256)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 6, 6, 384)         885120    
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 6, 6, 384)         1327488   
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 6, 6, 256)         884992    
__________

In [10]:
model2.fit(train_x,train_y,batch_size=50,epochs=20,verbose=1,validation_data=(test_x,test_y))
model2.save('model2.h5')

Train on 4000 samples, validate on 1000 samples
Epoch 1/20
4000/4000 [==============================] - 169s 42ms/step - loss: 0.7092 - acc: 0.5178 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 2/20
4000/4000 [==============================] - 168s 42ms/step - loss: 0.6953 - acc: 0.5283 - val_loss: 0.6900 - val_acc: 0.5020
Epoch 3/20
4000/4000 [==============================] - 176s 44ms/step - loss: 0.6865 - acc: 0.5558 - val_loss: 0.7353 - val_acc: 0.5000
Epoch 4/20
4000/4000 [==============================] - 171s 43ms/step - loss: 0.6923 - acc: 0.5510 - val_loss: 0.6758 - val_acc: 0.5410
Epoch 5/20
4000/4000 [==============================] - 172s 43ms/step - loss: 0.6890 - acc: 0.5510 - val_loss: 0.6700 - val_acc: 0.5730
Epoch 6/20
4000/4000 [==============================] - 173s 43ms/step - loss: 0.6918 - acc: 0.5560 - val_loss: 0.6644 - val_acc: 0.5880
Epoch 7/20
4000/4000 [==============================] - 169s 42ms/step - loss: 0.6964 - acc: 0.5530 - val_loss: 0.6692 - val_acc: 

### Model3: ZFNet

In [11]:
model3 = Sequential()  
model3.add(Conv2D(96,(7,7),strides=(2,2),input_shape=(64,64,3),padding='valid',activation='relu',kernel_initializer='uniform'))  
model3.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))  
model3.add(Conv2D(256,(5,5),strides=(2,2),padding='same',activation='relu',kernel_initializer='uniform'))  
model3.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))  
model3.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model3.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model3.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model3.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))  
model3.add(Flatten())  
model3.add(Dense(4096,activation='relu'))  
model3.add(Dropout(0.5))  
model3.add(Dense(4096,activation='relu'))  
model3.add(Dropout(0.5))  
#model3.add(Dense(1000,activation='softmax'))
model3.add(Dense(2,activation='softmax'))
model3.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])  
model3.summary()  

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 29, 29, 96)        14208     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 14, 14, 96)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 256)         614656    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 3, 3, 256)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 3, 3, 384)         885120    
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 3, 3, 384)         1327488   
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 3, 3, 256)         884992    
__________

In [12]:
model3.fit(train_x,train_y,batch_size=50,epochs=20,verbose=1,validation_data=(test_x,test_y))
model3.save('model3.h5')

Train on 4000 samples, validate on 1000 samples
Epoch 1/20
4000/4000 [==============================] - 83s 21ms/step - loss: 0.6960 - acc: 0.5072 - val_loss: 0.6890 - val_acc: 0.6140
Epoch 2/20
4000/4000 [==============================] - 82s 21ms/step - loss: 0.6914 - acc: 0.5213 - val_loss: 0.6866 - val_acc: 0.5090
Epoch 3/20
4000/4000 [==============================] - 83s 21ms/step - loss: 0.6914 - acc: 0.5270 - val_loss: 0.6811 - val_acc: 0.6340
Epoch 4/20
4000/4000 [==============================] - 82s 21ms/step - loss: 0.6849 - acc: 0.5568 - val_loss: 0.6734 - val_acc: 0.6240
Epoch 5/20
4000/4000 [==============================] - 83s 21ms/step - loss: 0.6781 - acc: 0.5720 - val_loss: 0.6638 - val_acc: 0.6390
Epoch 6/20
4000/4000 [==============================] - 84s 21ms/step - loss: 0.6746 - acc: 0.5730 - val_loss: 0.6527 - val_acc: 0.6430
Epoch 7/20
4000/4000 [==============================] - 83s 21ms/step - loss: 0.6702 - acc: 0.5910 - val_loss: 0.6603 - val_acc: 0.5940


### Model4: VGG-13

In [15]:
model4 = Sequential()  
model4.add(Conv2D(64,(3,3),strides=(1,1),input_shape=(64,64,3),padding='same',activation='relu',kernel_initializer='uniform'))  
model4.add(Conv2D(64,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model4.add(MaxPooling2D(pool_size=(2,2)))  
model4.add(Conv2D(128,(3,2),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model4.add(Conv2D(128,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model4.add(MaxPooling2D(pool_size=(2,2)))  
model4.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model4.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model4.add(MaxPooling2D(pool_size=(2,2)))  
model4.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model4.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model4.add(MaxPooling2D(pool_size=(2,2)))  
model4.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model4.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model4.add(MaxPooling2D(pool_size=(2,2)))  
model4.add(Flatten())  
model4.add(Dense(4096,activation='relu'))  
model4.add(Dropout(0.5))  
model4.add(Dense(4096,activation='relu'))  
model4.add(Dropout(0.5))  
model4.add(Dense(2,activation='softmax'))  
model4.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])  
model4.summary()  

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_23 (Conv2D)           (None, 64, 64, 64)        1792      
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 64, 64, 64)        36928     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 32, 32, 128)       49280     
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 32, 32, 128)       147584    
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 16, 16, 256)       295168    
__________

In [16]:
model4.fit(train_x,train_y,batch_size=50,epochs=20,verbose=1,validation_data=(test_x,test_y))
model4.save('model4.h5')

Train on 4000 samples, validate on 1000 samples
Epoch 1/20
4000/4000 [==============================] - 544s 136ms/step - loss: 0.6930 - acc: 0.5110 - val_loss: 0.6928 - val_acc: 0.5360
Epoch 2/20
4000/4000 [==============================] - 538s 134ms/step - loss: 0.6930 - acc: 0.5038 - val_loss: 0.6923 - val_acc: 0.5270
Epoch 3/20
4000/4000 [==============================] - 538s 134ms/step - loss: 0.6924 - acc: 0.5212 - val_loss: 0.6921 - val_acc: 0.5200
Epoch 4/20
4000/4000 [==============================] - 540s 135ms/step - loss: 0.6926 - acc: 0.5150 - val_loss: 0.6916 - val_acc: 0.5010
Epoch 5/20
4000/4000 [==============================] - 537s 134ms/step - loss: 0.6917 - acc: 0.5307 - val_loss: 0.6907 - val_acc: 0.5290
Epoch 6/20
4000/4000 [==============================] - 538s 135ms/step - loss: 0.6914 - acc: 0.5260 - val_loss: 0.6895 - val_acc: 0.6060
Epoch 7/20
4000/4000 [==============================] - 538s 134ms/step - loss: 0.6910 - acc: 0.5348 - val_loss: 0.6915 - va

# Data Augmentation Conclusion on LeNet

### without data augmentation 

In [36]:
def start_out():
    model0 = Sequential()  
    model0.add(Conv2D(32,(5,5),strides=(1,1),input_shape=(64,64,3),padding='valid',activation='relu',kernel_initializer='uniform')) 
    model0.add(MaxPooling2D(pool_size=(2,2)))  
    model0.add(Conv2D(64,(5,5),strides=(1,1),padding='valid',activation='relu',kernel_initializer='uniform'))  
    model0.add(MaxPooling2D(pool_size=(2,2)))  
    model0.add(Flatten())  
    model0.add(Dense(100,activation='relu'))  
    model0.add(Dense(2,activation='softmax'))  
    model0.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])  
    model0.summary()
    return model0

In [14]:
train_images,test_images = get_data(1,0.8)
train_x = prep_data(train_images,prefix_dir)
test_x = prep_data(test_images,prefix_dir)
train_y = [1 if x[:3]=='dog' else 0 for x in train_images]
test_y = [1 if x[:3]=='dog' else 0 for x in test_images]
train_y = to_categorical(train_y)
test_y = to_categorical(test_y)
print('train shape:',train_x.shape,train_y.shape)
print('test shape:',test_x.shape,test_y.shape)

Processed 2000 of 20000
Processed 4000 of 20000
Processed 6000 of 20000
Processed 8000 of 20000
Processed 10000 of 20000
Processed 12000 of 20000
Processed 14000 of 20000
Processed 16000 of 20000
Processed 18000 of 20000
Processed 20000 of 20000
Processed 2000 of 5000
Processed 4000 of 5000
train shape: (20000, 64, 64, 3) (20000, 2)
test shape: (5000, 64, 64, 3) (5000, 2)


In [27]:
model0 = start_out()
model0.fit(train_x,train_y,batch_size=50,epochs=20,verbose=1,validation_data=(test_x,test_y))
model0.save('model0_motoddata.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 60, 60, 32)        2432      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 26, 26, 64)        51264     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 10816)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               1081700   
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 202       
Total para

### With Data augmentation

In [21]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os

datagen = ImageDataGenerator(
          rotation_range=0.2,
          width_shift_range=0.2,
          height_shift_range=0.2,
          shear_range=0.2,
          zoom_range=0.2,
          horizontal_flip=True,
          fill_mode='nearest')


prefix = "C:/Users/xzh/Documents/Course_related/Big Data/Final Poject/input/"
old_doc = prefix + 'train/'
new_doc = prefix + 'augmentation'

# the .flow() command below generates batches of randomly transformed images and saves the results to the `preview/` directory
def generate_k_aug(file_path,to_path,k,prefix):
    img = load_img(file_path)  
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150 )   
    i = 0
    for batch in datagen.flow(x,batch_size=1,
                            save_to_dir=to_path,#the srore path of the picture
                            save_prefix=prefix,
                            save_format='jpg'):
        i += 1
        if i > k: 
             break  # otherwise the generator would loop indefinitely

In [22]:
# Add 3 new photo for each photo in original train_x and save all of them to new_doc
print(len(train_images))
if os.path.exists(new_doc):
    shutil.rmtree(new_doc)
os.mkdir(new_doc)
k=0
for f in train_images:
    k+=1
    if k%1000==0:
        print(k)
    prefix = f
    generate_k_aug(old_doc+f,new_doc,3,prefix)

20000
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000


In [32]:
# Transform the photo of new_doc into array style and add it into train_x and train_y
def get_new_data(new_doc):
    train_images = os.listdir(new_doc)
    train_x = prep_data(train_images,new_doc+'/')
    train_y = [1 if x[:3]=='dog' else 0 for x in train_images]
    train_y = to_categorical(train_y)
    return train_x,train_y
new_train_x,new_train_y = get_new_data(new_doc)
#train_x+=new_train_x
#train_y+=new_train_y
#print('train shape:',train_x.shape,train_y.shape)
#print('test shape:',test_x.shape,test_y.shape)

In [34]:
final_x = np.vstack((train_x,new_train_x))
final_y = np.vstack((train_y,new_train_y))
print('train shape:',final_x.shape,final_y.shape)
print('test shape:',test_x.shape,test_y.shape)

train shape: (60000, 64, 64, 3) (60000, 2)
test shape: (5000, 64, 64, 3) (5000, 2)


In [40]:
#model0 = start_out()
history = model0.fit(final_x,final_y,batch_size=50,epochs=6,verbose=1,validation_data=(test_x,test_y))
model0.save('model0_mottoddata.h5')

Train on 60000 samples, validate on 5000 samples
Epoch 1/6
60000/60000 [==============================] - 572s 10ms/step - loss: 0.5068 - acc: 0.7509 - val_loss: 0.4507 - val_acc: 0.7910
Epoch 2/6
60000/60000 [==============================] - 559s 9ms/step - loss: 0.4898 - acc: 0.7612 - val_loss: 0.4399 - val_acc: 0.7996
Epoch 3/6
60000/60000 [==============================] - 561s 9ms/step - loss: 0.4738 - acc: 0.7751 - val_loss: 0.4580 - val_acc: 0.7860
Epoch 4/6
60000/60000 [==============================] - 560s 9ms/step - loss: 0.4569 - acc: 0.7821 - val_loss: 0.4152 - val_acc: 0.8068
Epoch 5/6
60000/60000 [==============================] - 559s 9ms/step - loss: 0.4415 - acc: 0.7909 - val_loss: 0.4074 - val_acc: 0.8170
Epoch 6/6
60000/60000 [==============================] - 565s 9ms/step - loss: 0.4252 - acc: 0.8024 - val_loss: 0.3948 - val_acc: 0.8236


In [ ]:
history = model0.fit(final_x,final_y,batch_size=50,epochs=6,verbose=1,validation_data=(test_x,test_y))
model0.save('model0_mottoddata.h5')

# Model Ensemble Conclusion

### Calculate single model accuracy 

In [49]:
from keras.models import load_model  

In [50]:
model1 = load_model('model1.h5')
model2 = load_model('model2.h5')
model3 = load_model('model3.h5')
model4 = load_model('model4.h5')

In [53]:
left_perc = 1
train_perc = 1

train_images,test_images = get_data(left_perc,train_perc)
total_x = prep_data(train_images,prefix_dir)
total_y = [1 if x[:3]=='dog' else 0 for x in train_images]
total_y = to_categorical(total_y)
print('X shape:',total_x.shape)
print('Y shape:',total_y.shape)

X shape: (25000, 64, 64, 3)
Y shape: (25000, 2)


In [63]:
models = [model1,model2,model3,model4]
def calculate_accuracy(models,data_x,data_y):
    k=0
    for m in models:
        k+=1
        results = m.evaluate(data_x,data_y,verbose=0)
        print('The results for model{} is: \n     loss = {}, acc = {}\n'.format(k,results[0],results[1]))

calculate_accuracy(models,total_x,total_y)

The results for model1 is: 
     loss = 0.6027353811836242, acc = 0.66272

The results for model2 is: 
     loss = 0.6930758922576904, acc = 0.51116

The results for model3 is: 
     loss = 0.6582265794754029, acc = 0.62364

The results for model4 is: 
     loss = 0.6563407592010498, acc = 0.63092



### Calculate Ensemble Accuracy by different weights

In [67]:
def get_preds(models,data_x):
    preds = []
    for m in models:
        result = m.predict_classes(data_x)
        preds.append( to_categorical(result) )
    return preds

def preds_to_classes(preds,weights):
    score = np.zeros(preds[0].shape)
    for w,p in zip(weights,preds):
        score+=w*p
    classes = score.argmax(axis=1)
    return classes

def classes_to_accuracy(classes,data_y):
    accuracy = sum(classes==np.array([np.where(x==1) for x in data_y]).flatten())/len(data_y)
    return accuracy

preds = get_preds(models,total_x)

for weights in [[2,1,1,1],[3,1,1,1],[4,1,1,1]]:
    print('when weights = ',weights)
    classes = preds_to_classes(preds,weights)
    accuracy = classes_to_accuracy(classes,total_y)
    print('ensemble accuracy = ',accuracy,'\n')

when weights =  [2, 1, 1, 1]
ensemble accuracy =  0.66196 

when weights =  [3, 1, 1, 1]
ensemble accuracy =  0.66292 

when weights =  [4, 1, 1, 1]
ensemble accuracy =  0.66272 



# Thank you!